# Basis and reconstruction of 64 pixel image.
Joshua Stough
DIP

Decompose and partially reconstruct according to the Haar
basis.



In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
from waveletUtil import *


In [2]:

print('haarReconstructionDemo: Works only on 64 pixel images...')

I = plt.imread('surprise.png')
# I = plt.imread('strawberry.png')

# H is the 8x8 Haar matrix
# H = np.eye(8) # For fun comparison.
# H = makeRandomBasis(8)
# H = makeKLTBasis(I, 8)
# H = makeDCTMatrix(8)
H = makeHaarMatrix(8)

#The transform image, an image of coefficients wrt the Haar basis.
CT = np.zeros(I.shape)
for chan in range(3):
    CT[...,chan] = np.matmul(H, np.matmul(I[...,chan], H.transpose()))


#RI will represent the reconstructed image as we add back more
#Haar patterns
RI = np.zeros(I.shape) #Still should be 8x8


#Visual
fh, axh = plt.subplots(8,8, figsize=(8,8))
fr, axr = plt.subplots(8,8, figsize=(8,8))


# We're going to reconstruct according to distance from the
# 0,0 (the first Haar basis, average calculator). Notice,
# this order is independent of the actual image data.
xs = np.meshgrid(np.arange(8), np.arange(8), indexing='ij')
coords = np.concatenate([np.expand_dims(c, axis=1) for c in
                         [x.ravel() for x in xs]], axis=1)
dists = np.sum(coords*coords, axis=1)
darg = np.argsort(dists) #sorts in increasing order

#If we were to use magnitude of the coefficient...need to ravel in
#column-major, like we set up coords.
# mags = T.ravel(order='F')
# darg = np.argsort(mags)
# darg.reverse()
#
#c = 1 #used to display the order of reconstruction.

for ind in darg:
    i,j = coords[ind] #coords[darg[x]] if x in range(len(darg))

    #Construct that Haar basis and display it
    Bij = np.outer(H[i, :], H[j, :])
    axh[i][j].imshow(Bij, cmap='gray', vmin=-.5, vmax=.5)
    axh[i][j].axes.get_xaxis().set_visible(False)
    axh[i][j].axes.get_yaxis().set_visible(False)
    # axh[i][j].set_title('c_%d_%d: %6.3f' % (i, j, T[i,j]))
    # https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.text
    axh[i][j].text(0, 6, r'$\pi:%6.3f$' % CT[i, j, :].sum(), fontsize=6, color='cyan')

    # Add the amount of that basis that was in the original image to
    # the running total, or reconstruction.
    # RI = RI + T[i, j] * Bij #2D
    for chan in range(3):
        RI[...,chan] += CT[i,j,chan]*Bij

    RIcopy = RI.copy().clip(0,1)
    axr[i][j].imshow(RIcopy)
    axr[i][j].axes.get_xaxis().set_visible(False)
    axr[i][j].axes.get_yaxis().set_visible(False)
    # axr[i][j].text(0, 6, '%d' % c, fontsize=10, color='cyan')
    #The order of reconstruction isn't that informative, but useful for debugging.
    #c += 1


plt.show()

fh.canvas.set_window_title('Haar Basis Images and Coefficients')
fh.tight_layout()

fr.canvas.set_window_title('Partial Reconstructions from Large Scale to Small')
fr.tight_layout()


haarReconstructionDemo: Works only on 64 pixel images...


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from skimage.transform import rescale, resize

def f(x,pos):
    Big_mem = {}
    x = 2**x
    pos = pos * (x*x) - 1
    pos = int(pos)
    image = plt.imread('eye.png')
    I = resize(image, (x, x),anti_aliasing=True)    
    
    
    H = makeHaarMatrix(x)
    #The transform image, an image of coefficients wrt the Haar basis.
    CT = np.zeros(I.shape)
    for chan in range(3):
        CT[...,chan] = np.matmul(H, np.matmul(I[...,chan], H.transpose()))
        #print(CT)

#RI will represent the reconstructed image as we add back more
#Haar patterns
    RI = np.zeros(I.shape) #Still should be 8x8

# We're going to reconstruct according to distance from the
# 0,0 (the first Haar basis, average calculator). Notice,
# this order is independent of the actual image data.
    xs = np.meshgrid(np.arange(x), np.arange(x), indexing='ij')
    coords = np.concatenate([np.expand_dims(c, axis=1) for c in [x.ravel() for x in xs]], axis=1)
    dists = np.sum(coords*coords, axis=1)
    darg = np.argsort(dists) #sorts in increasing order

#If we were to use magnitude of the coefficient...need to ravel in
#column-major, like we set up coords.
# mags = T.ravel(order='F')
# darg = np.argsort(mags)
# darg.reverse()
#
    c = 0 #used to display the order of reconstruction.

    for ind in darg:

        i,j = coords[ind] #coords[darg[x]] if x in range(len(darg))

        #Construct that Haar basis and display it
        Bij = np.outer(H[i, :], H[j, :])
        
        # Add the amount of that basis that was in the original image to
        # the running total, or reconstruction.
        # RI = RI + T[i, j] * Bij #2D
        for chan in range(3):
            RI[...,chan] += CT[i,j,chan]*Bij

        RIcopy = RI.copy().clip(0,1)
        #axr[i][j].imshow(RIcopy)
        #axr[i][j].axes.get_xaxis().set_visible(False)
        #axr[i][j].axes.get_yaxis().set_visible(False)
        #axr[i][j].text(0, 6, '%d' % c, fontsize=10, color='cyan')
        #The order of reconstruction isn't that informative, but useful for debugging.
        Big_mem[ind] = RIcopy.copy()
        c += 1
        #print(ind, c)
        if (c == pos):
            #save Ps?
            P = RIcopy.copy()
    
    return Big_mem

   # plt.figure()
   # plt.show()

    #fh.canvas.set_window_title('Haar Basis Images and Coefficients')
    #fh.tight_layout()

    #fr.canvas.set_window_title('Partial Reconstructions from Large Scale to Small')
    #fr.tight_layout()
    #plt.close()
    #fig1 = plt.figure()
    #plt.imshow(P)



#interact(f, x=(2,8), pos=(0.0,1.0,0.01));

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation


def update_line(num, data, line):
    line.set_data(data[..., :num])
    return line,

In [5]:
fig1 = plt.figure()

# Fixing random state for reproducibility
np.random.seed(19680801)

data = np.random.rand(2, 25)
l, = plt.plot([], [], 'r-')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.xlabel('x')
plt.title('test')
line_ani = animation.FuncAnimation(fig1, update_line, 25, fargs=(data, l),
                                   interval=50, blit=True)

# To save the animation, use the command: line_ani.save('lines.mp4')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
fig2 = plt.figure()

x = np.arange(-9, 10)
y = np.arange(-9, 10).reshape(-1, 1)
base = np.hypot(x, y)
ims = []
for add in np.arange(15):
    ims.append((plt.pcolor(x, y, base + add, norm=plt.Normalize(0, 30)),))

#im_ani = animation.ArtistAnimation(fig2, ims, interval=50, repeat_delay=3000,
                                   blit=True)
# To save this second animation with some metadata, use the following command:
# im_ani.save('im.mp4', metadata={'artist':'Guido'})

plt.show()

IndentationError: unexpected indent (<ipython-input-6-b5c682239d0e>, line 11)

In [10]:

fps = 30
nSeconds = 5
snapshots = f(5,2)# generates a bunch of 5x5 arrays
# First set up the figure, the axis, and the plot element we want to animate

fig = plt.figure( figsize=(8,8) )

a = snapshots[0] # inital pic 
im = plt.imshow(a, interpolation='none', aspect='auto', vmin=0, vmax=1) #show inital 

def animate_func(i):
    if i % len(snapshots) == 0: #keeps incrementing i for loop back
        print( '.', end ='' )

    im.set_array(snapshots[i]) # resets plt.imshow()
    return [im] #returns it as array?

anim = animation.FuncAnimation(
                               fig, 
                               animate_func, 
                             
                               interval = 1000 / 30, # in ms 
                               )

#anim.save('test_anim.mp4', fps=fps, extra_args=['-vcodec', 'libx264'])

print('Done!')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Done!
